In [35]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [36]:
# importing stuff
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

<IPython.core.display.Javascript object>

In [37]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

BASE_URL = "http://www.yellowpages.com"
num_iterations = 30
main_list = []

max_pages_to_scrape = 100

current_page  = 1

def restaurant(info_tag):
    """Takes a info tag as parameter 
    returns name of the restaurant"""
    output = info_tag.find('a', class_='business-name').text
    return output


def categories_list(info_tag):
    """ Returns a list of categories
    of the restaurent"""
    all_category_a_tags = info_tag.find("div",class_="categories").find_all('a')
    output = [a.text for a in all_category_a_tags]
    return output


def ta_ratings(info_tag):
    """Returns a dictionary of trip advisor ratings"""
    ta_tag = info_tag.find('div', class_="ratings")
    output = ta_tag["data-tripadvisor"] if ta_tag and "data-tripadvisor" in ta_tag.attrs else None
    return output

# def yp_ratings(info_tag):
#     """ Returns a dictionary of raitngs and count"""
#     yp_ratings_tag = info_tag.find("div", class_="ratings")
#     print(yp_ratings_tag)
#     if yp_ratings_tag:
#         yp_ratings_tag_for_a = yp_ratings_tag.find("a",class_="rating")
#         # yellow pages rating
#         yp_rate = yp_ratings_tag_for_a.div["class"]
#         rating_value = " and ".join(yp_rate) if len(yp_rate) != 1 else " ".join(yp_rate)
        
#         # yellow pages count
#         yp_count = yp_ratings_tag.a.span.text.strip("()")
        
#         output = {"ratings":rating_value, "count":yp_count}
#         return output
#     return None

def yp_ratings(info_tag):
    """Returns a dictionary of ratings and count"""
    yp_ratings_tag = info_tag.find("div", class_="ratings")
    if yp_ratings_tag:
        # Find the 'a' tag with class 'rating' under yp_ratings_tag
        yp_ratings_tag_for_a = yp_ratings_tag.find("a", class_="rating")

        if yp_ratings_tag_for_a:
            # yellow pages rating
            yp_rate = yp_ratings_tag_for_a.div["class"]
            rating_value = " and ".join(yp_rate) if len(yp_rate) != 1 else " ".join(yp_rate)

            # yellow pages count
            yp_count_tag = yp_ratings_tag_for_a.span
            yp_count = yp_count_tag.text.strip("()") if yp_count_tag else None

            output = {"ratings": rating_value, "count": yp_count}
            return output
    
    return None



def links(info_tag):
    """Returns links of website, directions and menu else None"""
    link_class = info_tag.find("div",class_="links")
    if link_class:
        # website
        website_a_tag = link_class.find("a",class_="track-visit-website") 
        website_link = BASE_URL + website_a_tag["href"] if website_a_tag else None   
        
        # directions
        directions_a_tag = link_class.find("a", class_="track-map-it directions")
        directions_link = BASE_URL + directions_a_tag["href"] if directions_a_tag else None
        
        # menu
        menu_a_tag = link_class.find("a",class_= "menu")
        menu_link = BASE_URL + menu_a_tag["href"] if menu_a_tag else None   
        
        return {
            'website_link': website_link,
            'directions_link': directions_link,
            'menu_link': menu_link }

    return None


def years_in_buss(info_tag):
    badges_class = info_tag.find("div", class_= "badges")
    if badges_class: 
        present = badges_class.find("div", class_="years-in-business")
        years_in_business = present.div.text if present else None
        
        return years_in_business
    return None

        
def years_with_yellow_pages(info_tag):
    badges_class = info_tag.find("div", class_= "badges")
    if badges_class:
        yp_class = badges_class.find("div", class_="years-with-yp")
        if yp_class:
            years_with_yp = yp_class.div.text if yp_class else None
            return years_with_yp     
        return None
    return None


def amenities(info_tag):
    """Returns a list of amenities"""
    
    amenities_tag = info_tag.find("div",class_="amenities")
    if amenities_tag:
        amenities_info = amenities_tag.find("div", class_="amenities-info").find_all("span")
        output = [span.text for span in amenities_info]   
        return output      
    
    return None


def description(info_tag):
    """ Returns description of the restaurant"""
    
    description_tag = info_tag.find("div", class_="snippet")
    output = description_tag.p.text  if description_tag else None
    
    return output


def online_order(info_tag):
    """ Returns True if online order is
    availabe in the restaurant"""
    
    online = info_tag.find('div',class_='listing-ctas')
    if online:
        is_online = online.find('a',class_='action order-online')
        status = True if is_online else False
    
        return status
    return None


def contact_number(info_tag):
    """Returns phone no of the restaurent"""
    
    phone_number = info_tag.find('div',class_='phones phone primary')
    
    return phone_number.text if phone_number else None


def street_address(info_tag):
    """ returns street address """
    
    street = info_tag.find('div',class_='street-address')
    
    return street.text if street else None


def location(info_tag):
    """ returns locality """
    
    local = info_tag.find('div',class_='locality')
    
    return local.text if local else None



def price_range_sign(info_tag):
    """ returns price range  """
    
    price = info_tag.find('div',class_='price-range')  
    if price:
        return price.text
    return None
        





for page_number in range(1, max_pages_to_scrape + 1):
    url = f"https://www.yellowpages.com/search?search_terms=Restaurants&geo_location_terms=Los%20Angeles%2C%20CA&page={page_number}"
    source = requests.get(url, headers=headers).text
    soup = BeautifulSoup(source, "html.parser")
    
#     result_tags = soup.find_all("div", class_="result")
    
#     for result in result_tags:
#         info_tags = result.find_all("div",class_= "info")
    
    info_tags = soup.find_all("div", class_= "info")
    
    print(f"Scraping page: {page_number}")
    
    
    for index, value in enumerate(info_tags, start=1):
            restaurant_name = restaurant(value)
            categories = categories_list(value)
            trip_advisor_ratings = ta_ratings(value)
            yellow_ratings_and_count = yp_ratings(value)
            various_links = links(value)
            total_business_years = years_in_buss(value)
            years_with_yp = years_with_yellow_pages(value)
            amenities_available = amenities(value)
            description_ = description(value)
            status = online_order(value)
            phone_number = contact_number(value)
            street = street_address(value)
            locality = location(value) 
            price_range = price_range_sign(value)            
    
            data = {'Name':restaurant_name,'Type':categories,'Trip_advisor_ratings':trip_advisor_ratings,'Yellow_page_ratings':yellow_ratings_and_count,'Website_link':various_links['website_link'],'Directions_link':various_links['directions_link'],'Menu_link':various_links['menu_link'],
            'Total_business_years':total_business_years, "Years_with_yellow_pages":years_with_yp, 'Amenities_available':amenities_available,'Description':description_,'Order_online':status,'Phone_Number':phone_number,'Street_address':street,"locality":locality,"Price_range":price_range}

            print(index,data)
            print()
            main_list.append(data)
    
#     next_button = soup.find('a', class_="next ajax-page")
#     if next_button:
#         current_page += 1
#     else:
#         print("No more pages. Exitingg....")
#         break


Scraping page: 1
1 {'Name': 'Palermo Ristorante Italiano', 'Type': ['Restaurants', 'Pizza', 'Italian Restaurants'], 'Trip_advisor_ratings': '{"rating":"4.5","count":"135"}', 'Yellow_page_ratings': {'ratings': 'result-rating and four and half', 'count': '44'}, 'Website_link': 'http://www.yellowpages.comhttp://palermorestaurant.net', 'Directions_link': 'http://www.yellowpages.com/listings/1001234798248/directions', 'Menu_link': 'http://www.yellowpages.com/los-angeles-ca/mip/palermo-ristorante-italiano-540709640?lid=1001234798248#open-menu', 'Total_business_years': '44 Years', 'Years_with_yellow_pages': '8 Years with', 'Amenities_available': ['Wheelchair accessible'], 'Description': "Absolutely delectable! The best pizza experience I've had. A crispy, thin crust topped with premium ingredients that burst with flavor in every bite. Perfectly balanced cheese…", 'Order_online': True, 'Phone_Number': '(323) 663-1178', 'Street_address': '1858 N Vermont Ave', 'locality': 'Los Angeles, CA 90027'

Scraping page: 2
1 {'Name': "Berri's Pizza Cafe", 'Type': ['Restaurants', 'Italian Restaurants', 'Pizza'], 'Trip_advisor_ratings': '{"rating":"3.5","count":"63"}', 'Yellow_page_ratings': None, 'Website_link': None, 'Directions_link': None, 'Menu_link': 'http://www.yellowpages.com/los-angeles-ca/mip/berris-pizza-cafe-2773952#open-menu', 'Total_business_years': '29 Years', 'Years_with_yellow_pages': None, 'Amenities_available': ['Offers outdoor seating', 'Wheelchair accessible', 'Good for groups', 'Has Wifi', 'Serves alcohol', 'Takes reservations'], 'Description': 'From Business: Founded in 1996 by Raphael Berry and his wife, Hortensia Magana, Berri\'s Cafe has been named ""TOP 25 BEST RESTAURANTS IN LOS ANGELES"" by Gayot.com for Spring…', 'Order_online': True, 'Phone_Number': '(323) 852-0642', 'Street_address': '8412 W 3rd St', 'locality': 'Los Angeles, CA 90048', 'Price_range': '$$'}

2 {'Name': 'Veggie Grill', 'Type': ['Restaurants', 'Vegetarian Restaurants', 'Breakfast, Brunch & Lun

Scraping page: 3
1 {'Name': "Fabrocini's-Beverly Glen", 'Type': ['Restaurants', 'Pizza', 'Italian Restaurants'], 'Trip_advisor_ratings': '{"rating":"4.5","count":"48"}', 'Yellow_page_ratings': {'ratings': 'result-rating and four', 'count': '1'}, 'Website_link': 'http://www.yellowpages.comhttp://www.fabrocinibeverlyglen.com', 'Directions_link': None, 'Menu_link': None, 'Total_business_years': '26 Years', 'Years_with_yellow_pages': None, 'Amenities_available': ['Offers outdoor seating', 'Wheelchair accessible', 'Serves alcohol', 'Takes reservations'], 'Description': 'Just had a very good italian dinner here. Portions are quite large which is good since the rearaurat is a bit pricey for what I consider an family style restaurant. The outdoor…', 'Order_online': False, 'Phone_Number': '(310) 475-7404', 'Street_address': '2960 N Beverly Glen Cir', 'locality': 'Los Angeles, CA 90077', 'Price_range': '$$'}

2 {'Name': 'Taix French Restaurant', 'Type': ['Restaurants', 'French Restaurants', 'Fam

Scraping page: 4
1 {'Name': 'Chin Chin', 'Type': ['Restaurants', 'Chinese Restaurants', 'Caterers'], 'Trip_advisor_ratings': '{"rating":"3.5","count":"37"}', 'Yellow_page_ratings': {'ratings': 'result-rating and four', 'count': '1'}, 'Website_link': 'http://www.yellowpages.comhttp://www.chinchin.com', 'Directions_link': None, 'Menu_link': 'http://www.yellowpages.com/los-angeles-ca/mip/chin-chin-5303721#open-menu', 'Total_business_years': '38 Years', 'Years_with_yellow_pages': None, 'Amenities_available': ['Offers outdoor seating', 'Wheelchair accessible', 'Serves alcohol', 'Takes reservations'], 'Description': 'I love the food.  We always have our fave fried rice delivered.  Other entrees are just usual Chinese items. The restaurant has a lot of seating availabel, including outdoor…', 'Order_online': True, 'Phone_Number': '(310) 826-2525', 'Street_address': '11740 San Vicente Blvd', 'locality': 'Los Angeles, CA 90049', 'Price_range': '$$'}

2 {'Name': 'Momo Korean Bar-B-Q', 'Type': ['R

Scraping page: 5
1 {'Name': 'Fundamental La', 'Type': ['Restaurants', 'American Restaurants', 'Caterers'], 'Trip_advisor_ratings': '{"rating":"4.5","count":"54"}', 'Yellow_page_ratings': {'ratings': 'result-rating and five', 'count': '8'}, 'Website_link': 'http://www.yellowpages.comhttps://www.fundamental-la.com/tipplerclub-about', 'Directions_link': None, 'Menu_link': 'http://www.yellowpages.com/los-angeles-ca/mip/fundamental-la-466661802#open-menu', 'Total_business_years': '18 Years', 'Years_with_yellow_pages': None, 'Amenities_available': ['Serves alcohol', 'Takes reservations'], 'Description': "I've been here for both dinner and lunch. During lunch hours, it's a nice sandwich shop where you go order at the counter and they bring the food to your table. I had the…", 'Order_online': True, 'Phone_Number': '(310) 444-7581', 'Street_address': '1303 Westwood Blvd', 'locality': 'Los Angeles, CA 90024', 'Price_range': '$$'}

2 {'Name': 'Four Cafe', 'Type': ['Restaurants', 'American Restaur

...

Scraping page: 92
1 {'Name': 'C W & Chris Fish & Shrimp', 'Type': ['Restaurants', 'Seafood Restaurants', 'Caterers'], 'Trip_advisor_ratings': None, 'Yellow_page_ratings': {'ratings': 'result-rating and five', 'count': '2'}, 'Website_link': 'http://www.yellowpages.comhttps://www.cwchris.com', 'Directions_link': None, 'Menu_link': None, 'Total_business_years': '31 Years', 'Years_with_yellow_pages': None, 'Amenities_available': ['Wheelchair accessible'], 'Description': 'Great customer and even better food the fish is perfectly seasoned and the chicken is so good hot sauce was not needed on either! So good strongly recommend ITS AMAZING ��������"', 'Order_online': True, 'Phone_Number': '(323) 750-9403', 'Street_address': '6512 S Western Ave', 'locality': 'Los Angeles, CA 90047', 'Price_range': '$'}

2 {'Name': 'Buffalo Wild Wings', 'Type': ['Restaurants', 'Chicken Restaurants', 'Take Out Restaurants'], 'Trip_advisor_ratings': '{"rating":"3.0","count":"14"}', 'Yellow_page_ratings': {'rating

Scraping page: 93
1 {'Name': 'Albae Restaurant', 'Type': ['Restaurants', 'Family Style Restaurants'], 'Trip_advisor_ratings': None, 'Yellow_page_ratings': None, 'Website_link': None, 'Directions_link': None, 'Menu_link': None, 'Total_business_years': '17 Years', 'Years_with_yellow_pages': None, 'Amenities_available': None, 'Description': None, 'Order_online': True, 'Phone_Number': '(213) 488-1005', 'Street_address': '830 Maple Ave', 'locality': 'Los Angeles, CA 90014', 'Price_range': None}

2 {'Name': 'Planet Restaurant', 'Type': ['Restaurants', 'Family Style Restaurants', 'Mexican Restaurants'], 'Trip_advisor_ratings': None, 'Yellow_page_ratings': None, 'Website_link': 'http://www.yellowpages.comhttp://www.planetdailies.com', 'Directions_link': None, 'Menu_link': None, 'Total_business_years': '20 Years', 'Years_with_yellow_pages': None, 'Amenities_available': ['Offers outdoor seating', 'Good for families'], 'Description': None, 'Order_online': False, 'Phone_Number': '(213) 481-2088', 

Scraping page: 94
1 {'Name': 'Tortas Ahogadas El Guero Restaurant', 'Type': ['Restaurants', 'Mexican Restaurants', 'Caterers'], 'Trip_advisor_ratings': None, 'Yellow_page_ratings': None, 'Website_link': 'http://www.yellowpages.comhttp://tortasahogadaselguero.com', 'Directions_link': None, 'Menu_link': None, 'Total_business_years': '23 Years', 'Years_with_yellow_pages': None, 'Amenities_available': ['Offers outdoor seating', 'Wheelchair accessible'], 'Description': None, 'Order_online': False, 'Phone_Number': '(323) 262-8900', 'Street_address': '4508 Whittier Blvd', 'locality': 'Los Angeles, CA 90022', 'Price_range': '$'}

2 {'Name': 'Calimex Restaurant', 'Type': ['Restaurants', 'Family Style Restaurants', 'Mexican Restaurants'], 'Trip_advisor_ratings': None, 'Yellow_page_ratings': None, 'Website_link': 'http://www.yellowpages.comhttp://ordercalimexdeli.com', 'Directions_link': None, 'Menu_link': None, 'Total_business_years': None, 'Years_with_yellow_pages': None, 'Amenities_available':

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

<IPython.core.display.Javascript object>

In [38]:
main_list

[{'Name': 'Palermo Ristorante Italiano',
  'Type': ['Restaurants', 'Pizza', 'Italian Restaurants'],
  'Trip_advisor_ratings': '{"rating":"4.5","count":"135"}',
  'Yellow_page_ratings': {'ratings': 'result-rating and four and half',
   'count': '44'},
  'Website_link': 'http://www.yellowpages.comhttp://palermorestaurant.net',
  'Directions_link': 'http://www.yellowpages.com/listings/1001234798248/directions',
  'Menu_link': 'http://www.yellowpages.com/los-angeles-ca/mip/palermo-ristorante-italiano-540709640?lid=1001234798248#open-menu',
  'Total_business_years': '44 Years',
  'Years_with_yellow_pages': '8 Years with',
  'Amenities_available': ['Wheelchair accessible'],
  'Description': "Absolutely delectable! The best pizza experience I've had. A crispy, thin crust topped with premium ingredients that burst with flavor in every bite. Perfectly balanced cheese…",
  'Order_online': True,
  'Phone_Number': '(323) 663-1178',
  'Street_address': '1858 N Vermont Ave',
  'locality': 'Los Angel

<IPython.core.display.Javascript object>

In [39]:
df = pd.DataFrame.from_dict(main_list)


<IPython.core.display.Javascript object>

In [40]:
df

,Name,Type,Trip_advisor_ratings,Yellow_page_ratings,Website_link,Directions_link,Menu_link,Total_business_years,Years_with_yellow_pages,Amenities_available,Description,Order_online,Phone_Number,Street_address,locality,Price_range
0,Palermo Ristorante Italiano,"[Restaurants, Pizza, Italian Restaurants]","{""rating"":""4.5"",""count"":""135""}","{'ratings': 'result-rating and four and half',...",http://www.yellowpages.comhttp://palermorestau...,http://www.yellowpages.com/listings/1001234798...,http://www.yellowpages.com/los-angeles-ca/mip/...,44 Years,8 Years with,[Wheelchair accessible],Absolutely delectable! The best pizza experien...,True,(323) 663-1178,1858 N Vermont Ave,"Los Angeles, CA 90027",$
1,Casa Bianca Pizza Pie,"[Restaurants, Pizza, Italian Restaurants]","{""rating"":""4.5"",""count"":""107""}","{'ratings': 'result-rating and four and half',...",http://www.yellowpages.comhttp://www.casabianc...,None,http://www.yellowpages.com/los-angeles-ca/mip/...,68 Years,None,"[Wheelchair accessible, Good for groups, Serve...","Test#good food#good ambiance""",False,(323) 256-9617,None,"Los Angeles, CA 90041",$$
2,Oyabun Seafood,"[Restaurants, Seafood Restaurants, Asian Resta...",None,"{'ratings': 'result-rating and four', 'count':...",http://www.yellowpages.comhttps://www.oyabunse...,http://www.yellowpages.com/listings/1002166518...,None,5 Years,None,None,#highly recommended #reasonable prices #friend...,True,(323) 716-4355,3060 W Olympic Blvd #150,"Los Angeles, CA 90006",None
3,Dulan Soul Food Kitchen,"[Restaurants, Soul Food Restaurants, American ...","{""rating"":""4.0"",""count"":""1""}","{'ratings': 'result-rating and four', 'count':...",http://www.yellowpages.comhttp://www.dulans-sf...,None,None,28 Years,None,"[Offers outdoor seating, Wheelchair accessible]","tast good, awesome , nice place""",False,(323) 418-8527,None,"Los Angeles, CA 90047",$$
4,Mauro's Restaurant,"[Restaurants, Breakfast, Brunch & Lunch Restau...","{""rating"":""4.5"",""count"":""44""}","{'ratings': 'result-rating and five', 'count':...",http://www.yellowpages.comhttp://www.mauroscaf...,None,http://www.yellowpages.com/los-angeles-ca/mip/...,12 Years,None,"[Offers outdoor seating, Wheelchair accessible...","awweeeessomeee good food n""",True,(323) 653-7970,8112 Melrose Ave,"Los Angeles, CA 90046",$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,Full House Buffet,"[Restaurants, Buffet Restaurants, Asian Restau...",None,"{'ratings': 'result-rating and five', 'count':...",None,None,None,16 Years,None,None,"Great food- loved their soft shell crabs, temp...",False,(323) 261-9988,4917 Whittier Blvd,"Los Angeles, CA 90022",$$
2816,Johnny Rockets,"[Restaurants, American Restaurants, Family Sty...",None,"{'ratings': 'result-rating and four and half',...",None,None,None,37 Years,None,"[Wheelchair accessible, Serves alcohol]",★★★★★Johnny Rockets Tastes Amazing!I really en...,False,(310) 289-0400,100 N La Cienega Blvd,"Los Angeles, CA 90048",$$
2817,HongKong BBQ Restaurant,"[Restaurants, Barbecue Restaurants, Caterers]","{""rating"":""4.0"",""count"":""42""}","{'ratings': 'result-rating and four', 'count':...",http://www.yellowpages.comhttp://www.hongkongb...,None,None,34 Years,None,"[Wheelchair accessible, Serves alcohol]","They rebranded - though it's still Sam Woo, th...",False,(213) 687-7238,803 N Broadway,"Los Angeles, CA 90012",$$
2818,Dong Il Jang Restaurant,"[Restaurants, Family Style Restaurants]",None,{'ratings': 'result-rating and three and half'...,None,None,http://www.yellowpages.com/los-angeles-ca/mip/...,43 Years,None,"[Wheelchair accessible, Serves alcohol, Takes ...","Until Dong Il Jang, I was a Korean BBQ naysaye...",False,(213) 383-5757,3455 W 8th St,"Los Angeles, CA 90005",$$


<IPython.core.display.Javascript object>

In [41]:
df.to_csv("main_data.csv", index=False)

<IPython.core.display.Javascript object>